<a href="https://colab.research.google.com/github/SourCherries/FR-APSDEHAL/blob/master/FR_APSDEHAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face recognition CNN by APSDEHAL from

Minimal script to run forward pass on model using initial parameters. Uncertain whether initial paramters are random. Goal is to pinpoint source of error when running main.py where weights appear to be on cpu despite putting entire model onto gpu.

Based on [pytorch example](https://pytorch.org/hub/pytorch_vision_vgg/) and on [CNN by APSDEHAL](https://github.com/apsdehal/Face-Recognition).

In [1]:
!git clone https://github.com/SourCherries/FR-APSDEHAL.git

Cloning into 'FR-APSDEHAL'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 112 (delta 27), reused 0 (delta 0), pack-reused 64
Receiving objects: 100% (112/112), 207.43 KiB | 687.00 KiB/s, done.
Resolving deltas: 100% (55/55), done.


In [2]:
device_model = "cuda"
device_input = "cuda"

In [3]:
import sys
sys.path.insert(1, '/content/FR-APSDEHAL')

import torch
from PIL import Image
from torchvision import transforms
# from model_noargs import Network
from model_modlist import Network
from constants import IMG_SIZE, NUM_CHANNELS

from PIL import Image
from torchvision.io import read_image

# Load image(s)

In [4]:
# judi = read_image("/content/FR-APSDEHAL/judi-dench-256.png")
# kathy = read_image("/content/FR-APSDEHAL/kathleen-turner-256.png")
# input_batch = torch.cat([torch.unsqueeze(judi, 0), torch.unsqueeze(kathy, 0)], dim=0)

# print(input_batch.shape)
# print(input_batch.dtype)

torch.Size([2, 3, 256, 256])
torch.uint8


In [4]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

judi = Image.open("/content/FR-APSDEHAL/judi-dench-256.png")
kathy = Image.open("/content/FR-APSDEHAL/kathleen-turner-256.png")
input_tensor = torch.cat([torch.unsqueeze(preprocess(judi), 0), torch.unsqueeze(preprocess(kathy), 0)], dim=0)
input_tensor.shape

torch.Size([2, 3, 256, 256])

In [5]:

# input_image = Image.open("/content/FR-APSDEHAL/judi-dench-256.png")
# input_tensor = preprocess(input_image)
# input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

input_batch = input_tensor
input_batch = input_batch.to(device_input)

print(input_batch.shape)
print(input_batch.dtype)
print(input_batch.min())
print(input_batch.max())

torch.Size([2, 3, 256, 256])
torch.float32
tensor(-2.1179, device='cuda:0')
tensor(2.4286, device='cuda:0')


# Test model

In [6]:
model = Network().to(device_model)
# model = Network()
# model = model.to(device_model)
with torch.no_grad():
    output = model(input_batch)

torch.Size([2, 64, 58, 58])
torch.Size([2, 215296])


/content/FR-APSDEHAL/model_modlist.py:63: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.functional.log_softmax(self.fully_connected2(x))


In [ ]:
print(output.shape)
print(output.dtype)
print(output)

In [7]:
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([0.0672, 0.0653, 0.0660, 0.0661, 0.0653, 0.0665, 0.0654, 0.0676, 0.0686,
        0.0692, 0.0676, 0.0688, 0.0657, 0.0645, 0.0662], device='cuda:0')


In [8]:
model.conv_output_size

215296